## 测试程序：模拟数据集上的二分类问题

In [1]:
import tensorflow as tf
from numpy.random import RandomState

定义训练数据batch的大小

In [2]:
batch_size = 8 

定义神经网络的结构和参数

In [3]:
w1 = tf.Variable(tf.random_normal([2,3],stddev=1.0,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1.0,seed=1)) 

定义样本的输入和真实输出

In [4]:
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input') #数据量小时用None较为方便 数据量大时容易内存溢出

定义向前传播过程（运算）

In [5]:
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

定义损失函数

In [6]:
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y,1e-10,1.0)) + (1-y_)* tf.log(tf.clip_by_value(1-y,1e-10,1.0)))

定义训练过程

In [7]:
learning_rate = 0.001
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

生成模拟数据集

In [8]:
rdm = RandomState(1)
dataset_size = 128 
X = rdm.rand(dataset_size, 2)

定义规则给出样本的标签 此处所有x1+x2<1的样例都被认为是正样本

In [9]:
Y = [[int(x1+x2<1)] for (x1,x2) in X]

创建会话，对变量进行初始化

In [10]:
sess = tf.Session()

init_op = tf.global_variables_initializer()
sess.run(init_op)

查看训练前的神经网络参数

In [11]:
print(sess.run(w1))
print(sess.run(w2))

[[-0.81131822  1.48459876  0.06532937]
 [-2.4427042   0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]


进行训练

In [12]:
STEPS = 5000 #设定训练轮数
for i in range(STEPS):
    start = (i * batch_size) % dataset_size
    end = min(start+batch_size, dataset_size) # 每次选取batch_size个样本进行训练

    sess.run(train_step,
        feed_dict = {x: X[start:end], y_: Y[start:end]}) # 通过选取的样本进行训练，同时更新参数

    if i % 1000 == 0:
        total_cross_entropy = sess.run(
            cross_entropy, feed_dict = {x: X, y_: Y})
        print("After {:d} training step(s), cross entropy on all data is {:g}".format(i, total_cross_entropy))

After 0 training step(s), cross entropy on all data is 1.89805
After 1000 training step(s), cross entropy on all data is 0.655075
After 2000 training step(s), cross entropy on all data is 0.626172
After 3000 training step(s), cross entropy on all data is 0.615096
After 4000 training step(s), cross entropy on all data is 0.610309


查看训练后的神经网络参数

In [13]:
print(sess.run(w1))
print(sess.run(w2))

[[ 0.02476984  0.5694868   1.69219422]
 [-2.19773483 -0.23668921  1.11438966]]
[[-0.45544702]
 [ 0.49110931]
 [-0.9811033 ]]


保存日志文件并关闭会话

In [14]:
writer = tf.summary.FileWriter('c:/path/to/logs', tf.get_default_graph())
writer.close()
sess.close()